In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seoul2021.csv')
df.head(1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2021,1,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,626851391.0,25502,...,157,15667,7253,108,1098,2013,4344,7792,7568,28


In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import requests
import xml, xmltodict
import json
%matplotlib inline 

In [ ]:
#대분류(외식,서비스,소매)로 나누기
df.loc[df['서비스_업종_코드'].str.contains('CS1'),'분류'] = '외식업'
df.loc[df['서비스_업종_코드'].str.contains('CS2'),'분류'] = '서비스업'
df.loc[df['서비스_업종_코드'].str.contains('CS3'),'분류'] = '소매업'
df.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,분류
0,2021,1,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,626851391.0,25502,...,15667,7253,108,1098,2013,4344,7792,7568,28,외식업
1,2021,1,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,122957138.0,12039,...,6679,4293,52,926,1222,2698,3404,2670,9,외식업
2,2021,1,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,5093349.0,177,...,124,36,0,0,9,18,62,71,3,외식업
3,2021,1,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,13507733.0,1359,...,356,881,0,243,260,213,288,235,5,외식업
4,2021,1,A,골목상권,2110001,이북5도청사,CS200001,일반교습학원,37551380.0,128,...,56,72,0,0,8,96,8,16,3,서비스업


In [ ]:
#필요한 열만 가져오기

food = df[df['분류']=='외식업']

food = food[['상권_코드','상권_코드_명', '서비스_업종_코드_명', '분기당_매출_금액', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액','화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액','시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액','연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '점포수','분류']]
food['분류'].unique()

array(['외식업'], dtype=object)

In [ ]:
#분기별로 한 점포당 매출금액 구하기
div = ['분기당_매출_금액', '월요일_매출_금액','화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액','시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액','연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']

for i in div:
  food[i] = food[i] / food['점포수']

food.rename(columns={'분기당_매출_금액':'한_점포당_분기별_매출_금액'}, inplace=True)

food.head(2)

,상권_코드,상권_코드_명,서비스_업종_코드_명,한_점포당_분기별_매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,...,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,점포수,분류
0,2110001,이북5도청사,한식음식점,2.238755e+07,367617856.0,259233535.0,2.182818e+06,2.913635e+06,2.900728e+06,2.533008e+06,...,1.319211e+07,6.034029e+06,33387.785714,659016.392857,1.556685e+06,3.476915e+06,6.761393e+06,6.738743e+06,28,외식업
1,2110001,이북5도청사,분식전문점,1.366190e+07,61070094.0,61887044.0,9.568364e+05,1.696012e+06,1.181039e+06,1.718316e+06,...,7.112296e+06,5.297650e+06,27969.000000,517897.000000,1.123617e+06,2.790576e+06,5.361795e+06,2.588091e+06,9,외식업


In [ ]:
# Open API 데이터 가져오기
key = '546f7464586a756e3733666b687678'
url1 = 'http://openapi.seoul.go.kr:8088/{}/xml/TbgisTrdarRelm/1/1000/'.format(key)
url2 = 'http://openapi.seoul.go.kr:8088/{}/xml/TbgisTrdarRelm/1001/1671/'.format(key)

# 1 ~ 1000 행
response1 = requests.get(url1)
content1 = response1.content
dict1 = xmltodict.parse(content1)
jsonString1 = json.dumps(dict1['TbgisTrdarRelm']['row'], ensure_ascii= False)
jsonObj1 = json.loads(jsonString1)

# 1001 ~ 1672 행
response2 = requests.get(url2)
content2 = response2.content
dict2 = xmltodict.parse(content2)
jsonString2 = json.dumps(dict2['TbgisTrdarRelm']['row'], ensure_ascii= False)
jsonObj2 = json.loads(jsonString2)

# for item in jsonObj2:
#     print(item['TRDAR_SE_CD'], item['TRDAR_SE_CD_NM'], item['TRDAR_CD'],
#         item['TRDAR_CD_NM'],item['XCNTS_VALUE'], item['YDNTS_VALUE'],
#         item['SIGNGU_CD'], item['ADSTRD_CD'], item['STDR_YM_CD'])

list_1 = []
list_2 = []
list_3 = []

for item in jsonObj1:
    list_1.append(item['TRDAR_CD'])
    list_2.append(item['XCNTS_VALUE'])
    list_3.append(item['YDNTS_VALUE'])

for item in jsonObj2:
    list_1.append(item['TRDAR_CD'])
    list_2.append(item['XCNTS_VALUE'])
    list_3.append(item['YDNTS_VALUE'])



In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.8 MB 14.1 MB/s 


In [ ]:
from pyproj import Proj, transform

In [ ]:
# Projection 정의
proj_1 = Proj(init='epsg:5181')

# WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
proj_2 = Proj(init='epsg:4326')

list_4 = []
list_5 = []

for item in list_2:
    list_4.append(int(item))

for item in list_3:
    list_5.append(int(item))

x_, y_ = transform(proj_1, proj_2, list_4, list_5)

dict_for_dataframe = {'상권_코드':list_1, 'X': x_, 'Y': y_}
coordinate = pd.DataFrame(dict_for_dataframe)
coordinate

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-85-41145ca1da7a>:16: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x_, y_ = transform(proj_1, proj_2, list_4, list_5)


,상권_코드,X,Y
0,2110753,126.908844,37.448687
1,2110427,127.070978,37.634469
2,2110755,126.912978,37.451006
3,2110739,126.901564,37.441482
4,2111017,127.115614,37.509060
...,...,...,...
1666,1001493,127.011025,37.567312
1667,1001494,126.997374,37.569998
1668,1001495,127.115275,37.516476
1669,1001496,127.060059,37.511003


In [ ]:
# 전체 외식업 한점포당 1년 매출 TOP10
eatfood = food.drop(columns=['상권_코드','점포수','서비스_업종_코드_명'])
eatfood = eatfood.groupby(['상권_코드_명']).sum()
eatfood.rename(columns={'한_점포당_분기별_매출_금액':'외식업_연매출_금액'}, inplace=True)
eatfood.head()

,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,...,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액
상권_코드_명,,,,,,,,,,,,,,,,,,,,,
4.19민주묘지역 2번,4.122043e+08,5.074806e+09,3.190069e+09,4.365845e+07,5.135672e+07,5.273364e+07,4.568431e+07,5.864800e+07,7.285429e+07,8.726891e+07,...,1.725737e+08,3.798654e+07,2.266353e+08,1.722508e+08,2.597306e+06,5.515224e+07,6.652854e+07,8.507251e+07,1.069288e+08,8.260671e+07
63빌딩,3.907348e+08,8.919831e+09,3.216556e+09,5.752587e+07,5.696745e+07,5.921136e+07,5.563498e+07,5.486659e+07,5.646218e+07,5.006633e+07,...,7.968719e+07,1.177284e+07,1.674009e+08,1.227010e+08,1.426834e+06,4.143750e+07,1.076853e+08,7.316587e+07,3.788742e+07,2.849905e+07
DMC(디지털미디어시티),1.251771e+09,8.348779e+10,1.159614e+10,1.991759e+08,2.270928e+08,2.215268e+08,2.305169e+08,2.151906e+08,8.814911e+07,7.011874e+07,...,3.252123e+08,1.336445e+08,4.100654e+08,3.358330e+08,6.256755e+06,1.673500e+08,2.299935e+08,2.110270e+08,1.018148e+08,2.945637e+07
GS강동자이아파트,4.881138e+08,2.483070e+09,1.322897e+09,5.967703e+07,6.742865e+07,5.628591e+07,6.013578e+07,7.810087e+07,8.804023e+07,7.844535e+07,...,1.911297e+08,6.327048e+07,2.518722e+08,1.876911e+08,2.492833e+06,5.665300e+07,8.762309e+07,1.111311e+08,1.188163e+08,6.284693e+07
G타워,5.666744e+08,4.674009e+09,5.739245e+08,8.513741e+07,9.858214e+07,1.003136e+08,1.067211e+08,1.092430e+08,4.254189e+07,2.413519e+07,...,1.857765e+08,1.087500e+08,2.929081e+08,1.415984e+08,7.655191e+05,1.205939e+08,1.544198e+08,9.163787e+07,5.126273e+07,1.582672e+07


In [ ]:
# 전체 외식업 TOP10 상권 보기
total10 = eatfood.sort_values('외식업_연매출_금액',ascending=False).head(10)
total10

,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,...,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액
상권_코드_명,,,,,,,,,,,,,,,,,,,,,
강남 마이스 관광특구,5.527805e+09,7.256726e+10,3.095753e+10,6.709933e+08,7.098114e+08,7.639630e+08,7.764800e+08,8.801755e+08,9.229621e+08,8.034197e+08,...,2.123756e+09,1.112027e+08,2.037686e+09,2.525007e+09,5.102284e+07,1.368569e+09,1.486457e+09,8.644149e+08,5.258099e+08,2.664197e+08
삼성역,3.793065e+09,1.291144e+11,3.461033e+10,5.119195e+08,5.570578e+08,5.789105e+08,5.959601e+08,6.285989e+08,5.046042e+08,4.160139e+08,...,1.346135e+09,3.271272e+08,1.445644e+09,1.545293e+09,2.250751e+07,8.492628e+08,1.017208e+09,5.921340e+08,3.472807e+08,1.625433e+08
고속터미널(고속터미널역),3.578846e+09,3.752608e+10,1.714538e+10,4.309896e+08,4.224788e+08,4.764693e+08,4.724266e+08,5.675759e+08,6.508790e+08,5.580265e+08,...,1.321855e+09,1.747446e+08,1.451944e+09,1.893413e+09,3.912406e+07,9.512657e+08,8.690255e+08,6.793963e+08,5.299125e+08,2.766330e+08
경기고교사거리(경기고교앞사거리),3.451394e+09,1.472292e+10,4.445941e+09,4.886747e+08,4.678612e+08,5.575893e+08,5.261019e+08,5.570839e+08,4.575849e+08,3.964976e+08,...,1.521583e+09,6.145265e+08,1.183392e+09,7.882343e+08,3.977846e+06,1.703989e+08,3.843008e+08,4.248915e+08,4.833660e+08,5.046915e+08
잠실역,2.915209e+09,8.785881e+10,3.042163e+10,3.862629e+08,4.029484e+08,4.243240e+08,4.664172e+08,4.558468e+08,4.398801e+08,3.395298e+08,...,9.700513e+08,2.592650e+08,1.244547e+09,1.303922e+09,3.233687e+07,5.887727e+08,7.217856e+08,5.861680e+08,3.945035e+08,2.249022e+08
상봉역,2.806588e+09,3.736097e+10,2.076177e+10,3.180455e+08,3.193232e+08,3.474246e+08,3.279805e+08,4.054796e+08,5.616445e+08,5.266901e+08,...,1.123328e+09,3.071239e+08,1.481443e+09,1.280140e+09,7.120837e+07,7.205775e+08,6.665713e+08,5.830626e+08,4.589930e+08,2.611839e+08
신도림역,2.771879e+09,5.557451e+10,2.003865e+10,3.706302e+08,3.962268e+08,4.112362e+08,4.189480e+08,4.661008e+08,4.254680e+08,2.832691e+08,...,1.028832e+09,4.733682e+08,1.185939e+09,1.472557e+09,2.698834e+07,7.683821e+08,7.597162e+08,5.036854e+08,3.782795e+08,2.214446e+08
엔터식스 강남점,2.534534e+09,4.932030e+10,2.296069e+10,3.042138e+08,3.195246e+08,3.373534e+08,3.506566e+08,4.047815e+08,4.374383e+08,3.805658e+08,...,9.396052e+08,3.540417e+08,1.041474e+09,1.232206e+09,2.183479e+07,6.598541e+08,6.177189e+08,4.045778e+08,3.622464e+08,2.074483e+08
서울시청,2.422068e+09,5.493517e+10,8.351337e+09,3.469739e+08,4.153029e+08,4.144917e+08,4.296605e+08,4.277447e+08,2.190273e+08,1.688667e+08,...,7.217164e+08,3.200351e+08,9.099932e+08,7.803200e+08,6.126186e+06,3.721046e+08,5.032155e+08,3.815043e+08,3.184480e+08,1.089146e+08


In [ ]:
#분기별데이터 합치기(세부 외식업 별 한점포당 1년 매출)
yfood = food.drop(columns=['상권_코드','점포수'])
yfood = yfood.groupby(['서비스_업종_코드_명','상권_코드_명']).sum()
yfood.rename(columns={'한_점포당_분기별_매출_금액':'외식업_연매출_금액'}, inplace=True)
yfood.head()

외식업_연매출_금액      주중_매출_금액      주말_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                   
분식전문점       4.19민주묘지역 2번   9.973888e+07  4.904646e+08  4.086928e+08   
            63빌딩           2.205884e+07  3.305726e+08  2.204197e+07   
            DMC(디지털미디어시티)  1.476247e+08  1.683536e+10  2.264233e+09   
            GS강동자이아파트      3.013934e+07  1.027481e+08  2.439720e+07   
            G타워            6.685492e+07  6.872253e+08  6.643019e+07   

                              월요일_매출_금액     화요일_매출_금액     수요일_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                   
분식전문점       4.19민주묘지역 2번   3.781067e+06  1.154937e+07  1.391045e+07   
            63빌딩           3.954925e+06  4.263924e+06  4.522038e+06   
            DMC(디지털미디어시티)  2.521406e+07  2.693039e+07  2.652829e+07   
            GS강동자이아파트      3.274378e+06  8.022260e+06  4.843268e+06   
            G타워            1.192588e+07  1.226589e+07  1.153976e+07   

                              목요일_매출_금액     금요일_매출_금액     토요일_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                   
분식전문점       4.19민주묘지역 2번   1.182972e+07  1.203071e+07  1.850196e+07   
            63빌딩           4.149245e+06  3.916534e+06  6.951772e+05   
            DMC(디지털미디어시티)  2.725019e+07  2.419427e+07  9.376522e+06   
            GS강동자이아파트      2.671562e+06  6.240311e+06  2.263256e+06   
            G타워            1.328022e+07  1.204738e+07  4.545159e+06   

                              일요일_매출_금액  ...  시간대_17~21_매출_금액  \
서비스_업종_코드_명 상권_코드_명                      ...                    
분식전문점       4.19민주묘지역 2번   2.813559e+07  ...     3.531556e+07   
            63빌딩           5.569947e+05  ...     2.448464e+06   
            DMC(디지털미디어시티)  8.130978e+06  ...     4.151464e+07   
            GS강동자이아파트      2.824300e+06  ...     4.776299e+06   
            G타워            1.250630e+06  ...     1.974745e+07   

                           시간대_21~24_매출_금액      남성_매출_금액      여성_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                      
분식전문점       4.19민주묘지역 2번      9.698799e+05  5.467190e+07  4.116352e+07   
            63빌딩              5.396111e+04  7.273648e+06  5.810492e+06   
            DMC(디지털미디어시티)     2.930006e+06  4.840184e+07  4.587346e+07   
            GS강동자이아파트         4.095038e+05  1.084414e+07  5.879464e+06   
            G타워               1.319014e+06  3.767339e+07  2.052843e+07   

                            연령대_10_매출_금액  연령대_20_매출_금액  연령대_30_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                    
분식전문점       4.19민주묘지역 2번   404825.928571  9.960276e+06  1.469936e+07   
            63빌딩            12447.188889  2.520695e+06  4.252659e+06   
            DMC(디지털미디어시티)  752928.987261  2.285137e+07  2.909001e+07   
            GS강동자이아파트      149157.666667  2.067758e+06  3.308152e+06   
            G타워             53553.055556  1.846320e+07  2.163630e+07   

                           연령대_40_매출_금액  연령대_50_매출_금액  연령대_60_이상_매출_금액  
서비스_업종_코드_명 상권_코드_명                                                     
분식전문점       4.19민주묘지역 2번   2.035607e+07  2.496565e+07     2.544924e+07  
            63빌딩           3.619898e+06  2.209496e+06     4.689443e+05  
            DMC(디지털미디어시티)  2.579891e+07  1.283828e+07     2.943800e+06  
            GS강동자이아파트      3.382663e+06  5.527028e+06     2.288840e+06  
            G타워            1.161823e+07  5.358675e+06     1.071865e+06  

[5 rows x 24 columns]

In [ ]:
#food.set_index('상권_코드', inplace=True)
#coordinate.set_index('상권_코드', inplace=True)

KeyError: ignored

In [ ]:
#food, coordinate 데이터 join
final = pd.merge(food, coordinate, how='left',on='상권_코드')
final

ValueError: ignored

In [ ]:
final['상권_코드']

In [ ]:
#분기별데이터 합치기(한점포당 1년 매출)
food = food.drop(columns=['상권_코드','점포수'])
eatout = food.groupby('상권_코드_명').sum()
eatout.rename(columns={'한_점포당_분기별_매출_금액':'외식업_연매출_금액'}, inplace=True)
eatout

,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,...,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액
상권_코드_명,,,,,,,,,,,,,,,,,,,,,
4.19민주묘지역 2번,4.122043e+08,5.074806e+09,3.190069e+09,4.365845e+07,5.135672e+07,5.273364e+07,4.568431e+07,5.864800e+07,7.285429e+07,8.726891e+07,...,1.725737e+08,3.798654e+07,2.266353e+08,1.722508e+08,2.597306e+06,5.515224e+07,6.652854e+07,8.507251e+07,1.069288e+08,8.260671e+07
63빌딩,3.907348e+08,8.919831e+09,3.216556e+09,5.752587e+07,5.696745e+07,5.921136e+07,5.563498e+07,5.486659e+07,5.646218e+07,5.006633e+07,...,7.968719e+07,1.177284e+07,1.674009e+08,1.227010e+08,1.426834e+06,4.143750e+07,1.076853e+08,7.316587e+07,3.788742e+07,2.849905e+07
DMC(디지털미디어시티),1.251771e+09,8.348779e+10,1.159614e+10,1.991759e+08,2.270928e+08,2.215268e+08,2.305169e+08,2.151906e+08,8.814911e+07,7.011874e+07,...,3.252123e+08,1.336445e+08,4.100654e+08,3.358330e+08,6.256755e+06,1.673500e+08,2.299935e+08,2.110270e+08,1.018148e+08,2.945637e+07
GS강동자이아파트,4.881138e+08,2.483070e+09,1.322897e+09,5.967703e+07,6.742865e+07,5.628591e+07,6.013578e+07,7.810087e+07,8.804023e+07,7.844535e+07,...,1.911297e+08,6.327048e+07,2.518722e+08,1.876911e+08,2.492833e+06,5.665300e+07,8.762309e+07,1.111311e+08,1.188163e+08,6.284693e+07
G타워,5.666744e+08,4.674009e+09,5.739245e+08,8.513741e+07,9.858214e+07,1.003136e+08,1.067211e+08,1.092430e+08,4.254189e+07,2.413519e+07,...,1.857765e+08,1.087500e+08,2.929081e+08,1.415984e+08,7.655191e+05,1.205939e+08,1.544198e+08,9.163787e+07,5.126273e+07,1.582672e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
후암시장,6.269063e+08,4.511567e+09,9.185358e+08,8.261659e+07,9.067370e+07,9.049489e+07,9.832346e+07,1.125729e+08,8.320492e+07,6.901987e+07,...,2.156998e+08,7.531084e+07,2.205222e+08,2.597829e+08,2.188652e+06,6.515313e+07,1.234113e+08,1.242224e+08,1.086649e+08,5.666465e+07
휘문고교사거리,1.216875e+09,5.615522e+09,1.368762e+09,1.613203e+08,1.729387e+08,1.803380e+08,1.638698e+08,1.942378e+08,1.838094e+08,1.603614e+08,...,4.656722e+08,1.246166e+08,4.462173e+08,2.634043e+08,7.113772e+05,5.333325e+07,1.588924e+08,1.816037e+08,1.755092e+08,1.395717e+08
흑리단길,4.103162e+08,5.135324e+09,2.028775e+09,4.887002e+07,5.404234e+07,5.802933e+07,5.871851e+07,7.462250e+07,6.885387e+07,4.717969e+07,...,1.712848e+08,6.001486e+07,1.823116e+08,1.921525e+08,1.810447e+06,1.011929e+08,8.223647e+07,7.539074e+07,6.859510e+07,4.523853e+07


In [ ]:
eatout.sort_values('외식업_연매출_금액', ascending=False).index[:20]


Index(['강남 마이스 관광특구', '삼성역', '고속터미널(고속터미널역)', '경기고교사거리(경기고교앞사거리)', '잠실역',
       '상봉역', '신도림역', '엔터식스 강남점', '서울시청', '평화시장(통일상가, 동화상가)', '학동사거리', '대치역',
       '명동 남대문 북창동 다동 무교동 관광특구', '코엑스', '잠실 관광특구', '망우역 1번', '선릉역', '매봉역',
       '아차산역', '해방촌예술마을'],
      dtype='object', name='상권_코드_명')

In [ ]:
eatout.corr()

,점포당_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,...,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액
점포당_연매출_금액,1.000000,0.224209,0.271648,0.961018,0.971875,0.981915,0.978775,0.983140,0.948979,0.895991,...,0.943351,0.606535,0.959800,0.926283,0.480356,0.783648,0.929967,0.923535,0.899045,0.776962
주중_매출_금액,0.224209,1.000000,0.848947,0.242786,0.254921,0.245788,0.253674,0.226518,0.166074,0.133025,...,0.212389,0.165252,0.216976,0.161723,0.073388,0.188842,0.216368,0.162900,0.142707,0.126609
주말_매출_금액,0.271648,0.848947,1.000000,0.254864,0.261561,0.262291,0.264753,0.258731,0.275940,0.245243,...,0.281798,0.199050,0.281627,0.237354,0.147431,0.284199,0.280110,0.205364,0.197188,0.178743
월요일_매출_금액,0.961018,0.242786,0.254864,1.000000,0.965707,0.962823,0.963876,0.947538,0.855317,0.787562,...,0.878816,0.577060,0.902205,0.868826,0.435062,0.707809,0.864520,0.883193,0.856042,0.759419
화요일_매출_금액,0.971875,0.254921,0.261561,0.965707,1.000000,0.981585,0.984733,0.965605,0.865258,0.788856,...,0.890883,0.596571,0.916472,0.876995,0.438494,0.719687,0.886545,0.893086,0.861418,0.752165
수요일_매출_금액,0.981915,0.245788,0.262291,0.962823,0.981585,1.000000,0.987690,0.971832,0.890143,0.819001,...,0.912289,0.607218,0.923631,0.885481,0.436217,0.733173,0.894027,0.893754,0.866934,0.760836
목요일_매출_금액,0.978775,0.253674,0.264753,0.963876,0.984733,0.987690,1.000000,0.971960,0.880442,0.805345,...,0.906921,0.611173,0.920437,0.877292,0.426649,0.727319,0.893010,0.888434,0.859971,0.751866
금요일_매출_금액,0.983140,0.226518,0.258731,0.947538,0.965605,0.971832,0.971960,1.000000,0.911911,0.840708,...,0.926424,0.625341,0.939645,0.901501,0.446738,0.755378,0.912199,0.902070,0.885653,0.759608
토요일_매출_금액,0.948979,0.166074,0.275940,0.855317,0.865258,0.890143,0.880442,0.911911,1.000000,0.947668,...,0.930389,0.570322,0.939895,0.924955,0.517026,0.814565,0.920606,0.892637,0.872820,0.741189
일요일_매출_금액,0.895991,0.133025,0.245243,0.787562,0.788856,0.819001,0.805345,0.840708,0.947668,1.000000,...,0.882903,0.488314,0.899646,0.878580,0.513387,0.788020,0.867962,0.850746,0.833526,0.698500
